In [297]:
import pandas as pd
import nltk
import numpy as np

In [298]:
data = pd.read_csv('all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [299]:
data = data[['reviewText','rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [300]:
data.isnull().any()

reviewText    False
rating        False
dtype: bool

In [301]:
data['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [302]:
data['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [303]:
#preprocess and cleaning
rate = []
for i in data['rating']:
    if (i<3):
        rate.append(0)
    else:
        rate.append(1)
data['rating'] = rate
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [304]:
data['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [305]:
##lower
##stopwords
##stemming
##tokenization

In [306]:
data['reviewText'] = data['reviewText'].str.lower()

In [307]:
import re
temp = []
for i in data['reviewText']:
    temp.append(re.sub('[^a-z A-Z 0-9]',' ',i))
data['reviewText'] = temp

In [308]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopword = stopwords.words('english')

In [309]:
word = []
for i in data['reviewText']:
    words = word_tokenize(i)
    temp = [i for i in words if i not in stopword]
    word.append(' '.join(temp))
data['reviewText'] = word

In [310]:
from bs4 import BeautifulSoup

In [311]:
## Remove url 
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
data['reviewText']=data['reviewText'].apply(lambda x: " ".join(x.split()))

In [312]:
data.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four books expecting 34 con...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [313]:
from nltk.stem import WordNetLemmatizer
lemitizer = WordNetLemmatizer()

In [314]:
word = []
for i in data['reviewText']:
    temp = word_tokenize(i)
    temp = [lemitizer.lemmatize(j) for j in temp]
    word.append(' '.join(temp))
data['reviewText'] = word

In [315]:
data.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four book expecting 34 conc...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [316]:
#Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['reviewText'],data['rating'])

In [317]:
X_train

11481    never read book like steve peek long claw thri...
10025    whew book detail going give anything away scen...
7290     book moved rapidly ending predictiblebut reall...
8355     first book shelli stevens read fell love abby ...
2846     dylan sasha two wounded soul feel place red ro...
                               ...                        
5512     resist potential plot started k went downhill ...
6731     wow really say pleasantly surprised book came ...
8746     woman went halloween party nightclub owned vam...
5302     good read kept attention know going happen her...
4739     awful book bad start skipping stop reading man...
Name: reviewText, Length: 9000, dtype: object

In [318]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_arr = bow.fit_transform(X_train).toarray()
X_test_arr = bow.transform(X_test).toarray()

In [319]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tf = tfidf.fit_transform(X_train).toarray()
X_test_tf = tfidf.transform(X_test).toarray()

In [320]:
import gensim
words_train = []
words_test = []

for i in X_train:
    i = word_tokenize(i)
    words_train.append(i)
for i in X_test:
    i = word_tokenize(i)
    words_test.append(i)

model = gensim.models.Word2Vec(words_train)

In [321]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

X_train_w2v = []
X_test_w2v = []
for i in words_train:
    X_train_w2v.append(avg_word2vec(i))
for i in words_test:
    X_test_w2v.append(avg_word2vec(i))

X_train_w2v_arr = np.array(X_train_w2v)
X_test_w2v_arr = np.array(X_test_w2v)

In [322]:
from sklearn.naive_bayes import GaussianNB
nb_bow = GaussianNB().fit(X_train_arr, y_train)
nb_itf = GaussianNB().fit(X_train_tf,y_train)
nb_w2v = GaussianNB().fit(X_train_w2v_arr,y_train)

In [323]:
from sklearn.metrics import classification_report
y_pred_bow = nb_bow.predict(X_test_arr)
y_pred_tf = nb_itf.predict(X_test_tf)
y_pred_w2v = nb_w2v.predict(X_test_w2v_arr)

In [324]:
classification_report(y_test,y_pred_bow)

'              precision    recall  f1-score   support\n\n           0       0.44      0.67      0.53      1050\n           1       0.75      0.54      0.63      1950\n\n    accuracy                           0.59      3000\n   macro avg       0.60      0.61      0.58      3000\nweighted avg       0.64      0.59      0.60      3000\n'

In [325]:
classification_report(y_test,y_pred_tf)

'              precision    recall  f1-score   support\n\n           0       0.44      0.64      0.52      1050\n           1       0.75      0.56      0.64      1950\n\n    accuracy                           0.59      3000\n   macro avg       0.59      0.60      0.58      3000\nweighted avg       0.64      0.59      0.60      3000\n'

In [326]:
classification_report(y_test, y_pred_w2v)

'              precision    recall  f1-score   support\n\n           0       0.51      0.73      0.60      1050\n           1       0.81      0.63      0.71      1950\n\n    accuracy                           0.66      3000\n   macro avg       0.66      0.68      0.66      3000\nweighted avg       0.71      0.66      0.67      3000\n'